In [1]:
import pandas as pd 
import numpy as np
# used for time series imaging 
from pyts.image import GramianAngularField, MarkovTransitionField, RecurrencePlot
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler
np.random.seed(42)
torch.manual_seed(42)

In [2]:
df = pd.read_csv('data_reg.csv') # read data
df.head(5) # display first 5 rows

,city,2011-01,2011-02,2011-03,2011-04,2011-05,2011-06,2011-07,2011-08,2011-09,...,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03,2025-04
0,北京,100,100.5,100.5000,100.600500,100.801701,100.902503,100.902503,100.902503,100.902503,...,202.657128,201.643842,200.232335,198.830709,197.836555,197.638719,196.848164,197.045012,196.650922,196.847573
1,天津,100,101.0,101.5050,101.708010,101.402886,101.200080,101.503680,101.605184,101.300369,...,144.255692,143.390158,141.669476,140.961129,141.384012,141.666780,142.091780,141.665505,142.090502,142.658864
2,石家庄,100,100.2,101.1018,101.304004,101.506612,101.709625,101.913044,101.913044,102.014957,...,166.094445,165.762256,165.264969,164.438644,164.274206,163.617109,163.453492,162.963131,162.800168,162.474568
3,太原,100,99.6,99.8988,100.098598,100.498992,100.800489,101.002090,101.002090,101.103092,...,133.016470,132.750437,133.148688,133.414986,133.548401,133.548401,133.949046,134.082995,134.351161,134.754215
4,呼和浩特,100,100.7,101.0021,101.810117,102.217357,102.421792,102.729057,102.831786,102.831786,...,152.322589,151.713299,150.803019,149.596595,148.549419,147.361023,146.476857,145.597996,144.870006,144.580266


In [3]:
def split_housing_price_data(df):
    '''
    split data to train and test array
    param:
        df: dataframe with columns ['city', 'y', '2011-01', ..., '2024-10'];
    return:
        train_ts: ndarray with shape [N, T], time series data for training;
        test_ts: ndarray with shape [N, T], time series data for testing;
        train_y: ndarray with shape [N,], label for training;
        test_y: ndarray with shape [N,], label for testing;
    '''
    # extract time series data to a NumPy array: shape = [N, T]
    ts_columns = [col for col in df.columns if '2011-01' <= col <= '2024-10']
    X = df[ts_columns].values
    
    # extract label to a NumPy array: shape = [N,]
    y_columns = [col for col in df.columns if '2024-11' <= col <= '2025-04']
    y = df[y_columns].values
    
    # extract city names to a NumPy array: shape = [N,]
    city = df['city'].values
    
    # split data into training and testing sets
    train_ts, test_ts, train_y, test_y, train_city, test_city = train_test_split(X, y, city, test_size=0.2,random_state=42, shuffle=True)

    return train_ts, test_ts, train_y, test_y, train_city, test_city

# split the data into training and testing sets
train_ts, test_ts, train_y, test_y, train_city, test_city = split_housing_price_data(df)
# display the shapes of the resulting arrays
train_ts.shape, test_ts.shape, train_y.shape, test_y.shape

((56, 166), (14, 166), (56, 6), (14, 6))

In [4]:
def image_time_series(ts):
    '''
    Convert time series data into images
    param:
        ts: ndarray with shape [N, T], time series;
    return:
        images: ndarray with shape [N, 3, T, T], 3 channels image of time series;
    '''
    # create GAF images 
    gaf = GramianAngularField()
    gaf_image = gaf.fit_transform(ts) # shape [N, T, T]

    # create MTF images
    mtf = MarkovTransitionField(strategy='uniform')
    mtf_image = mtf.fit_transform(ts) # shape [N, T, T]

    # create RP images
    rp = RecurrencePlot(threshold='point')
    rp_image = rp.fit_transform(ts) # shape [N, T, T]

    # stack the images along the channel dimension
    images = np.stack([gaf_image, mtf_image, rp_image], axis=1) #  shape [N, 3, T, T]
    
    return images

train_X = image_time_series(train_ts)
test_X = image_time_series(test_ts)
train_X.shape, test_X.shape

((56, 3, 166, 166), (14, 3, 166, 166))

In [5]:
# to DataLoader
def prepare_loader(X, y, batch_size=64, shuffle=True, fit_scaler=True, scaler_y=None):
    '''
    Prepare DataLoader for training and testing
    param:
        X: ndarray with shape [N, C, T, T], images;
        y: ndarray with shape [N,], labels;
        batch_size: int, size of each batch;
    return:
        loader: DataLoader object;
    '''
    # transform X into torch.tensor
    X_tensor = torch.tensor(X, dtype=torch.float32)
    
    if fit_scaler:
        scaler_y = StandardScaler()
        y_scaled = scaler_y.fit_transform(y)
    else:
        y_scaled = scaler_y.transform(y)

    # transform y into torch.tensor
    y_tensor = torch.tensor(y_scaled, dtype=torch.float32)

    # bundle X_tensor and y_tensor into a TensorDataset
    dataset = TensorDataset(X_tensor, y_tensor)
    
    # transform the dataset into a DataLoader
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

    return loader, scaler_y

train_loader, scaler_y = prepare_loader(train_X, train_y, shuffle=True, fit_scaler=True)
test_loader, _ = prepare_loader(test_X, test_y, shuffle=False, fit_scaler=False, scaler_y=scaler_y)

In [6]:
class BasicBlock(nn.Module):
    # 定义扩展因子，用于调整输出通道数
    expansion = 1

    def __init__(self, in_channels, out_channels, stride=1):
        """
        初始化 BasicBlock 模块。

        参数:
        - in_channels: 输入特征图的通道数
        - out_channels: 输出特征图的通道数
        - stride: 卷积步长，默认为 1
        """
        super(BasicBlock, self).__init__()

        # 第一个卷积层：3x3 卷积，用于提取特征
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        # 第一个批归一化层：对卷积后的输出进行归一化
        self.bn1 = nn.BatchNorm2d(out_channels)

        # 第二个卷积层：3x3 卷积，用于进一步提取特征
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        # 第二个批归一化层：对卷积后的输出进行归一化
        self.bn2 = nn.BatchNorm2d(out_channels)

        # 定义 shortcut 连接（残差连接）
        self.shortcut = nn.Sequential()  # 默认 shortcut 是恒等映射（identity mapping）

        # 如果步长不为 1 或输入通道数与输出通道数不匹配，则需要调整 shortcut
        if stride != 1 or in_channels != self.expansion * out_channels:
            self.shortcut = nn.Sequential(
                # 1x1 卷积，用于调整通道数和空间尺寸
                nn.Conv2d(in_channels, self.expansion * out_channels, kernel_size=1, stride=stride, bias=False),
                # 批归一化层
                nn.BatchNorm2d(self.expansion * out_channels)
            )

    def forward(self, x):
        # 第一层卷积 + 批归一化 + ReLU 激活
        out = F.relu(self.bn1(self.conv1(x)))

        # 第二层卷积 + 批归一化
        out = self.bn2(self.conv2(out))

        # 将 shortcut 的输出与主路径的输出相加（残差连接）
        out += self.shortcut(x)

        # 对相加后的结果应用 ReLU 激活
        out = F.relu(out)

        return out
    
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=6):
        """
        初始化 ResNet 模型。

        参数:
        - block: 残差块类型（如 BasicBlock）
        - num_blocks: 每个阶段的残差块数量（如 [2, 2, 2, 2] 对应 ResNet-18）
        - num_classes: 分类任务的类别数
        """
        super(ResNet, self).__init__()

        # 初始通道数
        self.in_channels = 64

        # 初始卷积层：3x3 卷积，用于提取初步特征
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # 构建 4 个阶段的残差块堆叠
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)  # 第一阶段
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)  # 第二阶段
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)  # 第三阶段
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)  # 第四阶段

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))  # 全局平均池化层
        self.linear = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, out_channels, num_blocks, stride):
        """
        构建一个阶段的残差块堆叠。

        参数:
        - block: 残差块类型（如 BasicBlock）
        - out_channels: 输出通道数
        - num_blocks: 残差块数量
        - stride: 第一个残差块的步长

        返回:
        - nn.Sequential: 一个阶段的残差块堆叠
        """
        # 定义每个残差块的步长列表
        # 第一个残差块使用指定的 stride，后续残差块使用 stride=1
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []

        # 构建每个残差块
        for stride in strides:
            # 添加一个残差块
            layers.append(block(self.in_channels, out_channels, stride))
            # 更新输入通道数
            self.in_channels = out_channels * block.expansion

        # 将残差块堆叠成一个序列
        return nn.Sequential(*layers)

    def forward(self, x):
        """
        前向传播函数。

        参数:
        - x: 输入图像

        返回:
        - out: 分类结果
        """
        # 初始卷积层 + 批归一化 + ReLU 激活
        out = self.relu(self.bn1(self.conv1(x))) # (56, 64, 83, 83)
        out = self.maxpool(out) # (56, 64, 42, 42)

        # 通过 4 个阶段的残差块堆叠
        out = self.layer1(out)  # (56, 64, 42, 42)
        out = self.layer2(out)  # (56, 128, 21, 21)
        out = self.layer3(out)  # (56, 256, 11, 11)
        out = self.layer4(out)  # (56, 512, 6, 6)

        # 全局平均池化：将特征图的空间维度压缩为 1x1
        out = self.avgpool(out) # (56, 512, 1, 1)

        # 展平为向量
        out = torch.flatten(out, 1) # (56, 512)

        # 全连接层：将特征向量映射到类别数量
        out = self.linear(out)

        return out
    
def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

In [ ]:
def train(model, train_loader, criterion, optimizer, num_epochs, device, scaler_y):
    model.train()
    loss_list = []
    mse_list = []

    for epoch in range(num_epochs):
        epoch_loss = []
        all_preds = []
        all_targets = []

        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)

            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

            epoch_loss.append(loss.item())
            all_preds.append(output.detach().cpu())
            all_targets.append(target.detach().cpu())

        # 记录该 epoch 的平均 loss
        avg_loss = sum(epoch_loss) / len(epoch_loss)
        loss_list.append(avg_loss)

        # 拼接预测与真实值
        all_preds = torch.cat(all_preds, dim=0).numpy()
        all_targets = torch.cat(all_targets, dim=0).numpy()

        # inverse_transform 回原始尺度
        all_preds_real = scaler_y.inverse_transform(all_preds)
        all_targets_real = scaler_y.inverse_transform(all_targets)

        # 计算原始尺度下的 MSE
        mse = np.mean((all_preds_real - all_targets_real) ** 2)
        mse_list.append(mse)

        print(f'Epoch {epoch}, Avg Loss: {avg_loss:.4f}, real MSE: {mse:.4f}')

    return loss_list, mse_list

In [8]:
def test(model, test_loader, criterion, device, scaler_y):
    model.eval()
    total_loss = 0.0
    all_preds = []
    all_targets = []

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            total_loss += loss.item() * data.size(0)

            all_preds.append(output.cpu())
            all_targets.append(target.cpu())

    all_preds = torch.cat(all_preds, dim=0).numpy()
    all_targets = torch.cat(all_targets, dim=0).numpy()

    # inverse_transform 回原始尺度
    all_preds_real = scaler_y.inverse_transform(all_preds)
    all_targets_real = scaler_y.inverse_transform(all_targets)

    # 计算真实房价单位下的 MSE
    mse = np.mean((all_preds_real - all_targets_real) ** 2)

    print(f"Test MSE (real unit): {mse:.4f}")
    return all_preds_real, all_targets_real, mse

In [18]:
# 实例化 ResNet-18 模型
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ResNet18()

# 定义损失函数和优化器
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
# 学习率调度器
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

loss_list, mse_list = train(model, train_loader, criterion, optimizer, 200, device=device, scaler_y=scaler_y)

TypeError: train() got an unexpected keyword argument 'scaler_y'

In [ ]:
preds, targets, test_mse = test(model, test_loader, criterion, device, scaler_y=scaler_y)
test_mse

Test MSE (real unit): 1192.0739


1192.0739

In [16]:
# 训练函数
def train(model, train_loader, criterion, optimizer, num_epochs, device):
    model.train()
    loss_list = []
    for epoch in range(num_epochs):
        loss_list.append([])
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            if batch_idx % 100 == 0:
                print(f'Epoch {epoch}, Batch {batch_idx}, Loss: {loss.item()}')
                loss_list[-1].append(loss.item())
    return loss_list

# 测试函数
def test(model, test_loader, criterion, device):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    print(f'Test Loss: {test_loss:.4f}, Accuracy: {accuracy:.2f}%')

In [17]:
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error

def arima_batch_forecast(X, y_true, forecast_horizon=6, arima_order=(1, 1, 1)):
    """
    对每一条时间序列使用ARIMA拟合，并预测未来6个月房价
    参数：
        X: ndarray, shape = [N, T]
        y_true: ndarray, shape = [N, 6]
        forecast_horizon: int, 默认为6
        arima_order: tuple, ARIMA阶数
    返回：
        y_pred: ndarray, shape = [N, 6]
        mse: float, 平均MSE
    """
    N = X.shape[0]
    y_pred = np.zeros((N, forecast_horizon))

    for i in range(N):
        series = X[i]
        model = ARIMA(series, order=arima_order)
        try:
            fitted = model.fit()
            forecast = fitted.forecast(steps=forecast_horizon)
            y_pred[i] = forecast
        except:
            # 如果模型拟合失败，使用最后一个观测值作为预测
            y_pred[i] = series[-1]
    
    mse = np.mean((y_pred - y_true) ** 2)
    return y_pred, mse

In [ ]:
train_pred, train_mse = arima_batch_forecast(train_ts, train_y)
test_pred, test_mse = arima_batch_forecast(test_ts, test_y)

print(f"Train MSE: {train_mse:.4f}")
print(f"Test MSE: {test_mse:.4f}")